# SQL
SQL can be a powerful asset as a Data Scientist. In this week's Notebook, we will explore how we can use SQLAlchemy and Pandas to efficiently use SQL databases in our projects.

In [ ]:
import sqlalchemy
from sqlalchemy import (Table, Column, Integer, String, Float, Date,
                        MetaData, create_engine)
from sqlalchemy.sql import select
from datetime import date

In [ ]:
# Setup a connection to our toy SQLite database
engine = create_engine('sqlite:///test.db')

   ### Example 1: SQLAlchemy as a SQL Engine
   This is just to show the "core" SQLAlchemy. I don't suggest you necessarily do things this way...

In [ ]:
metadata = MetaData()

# Define SQL Schema
students = sqlalchemy.Table('student', metadata,
    Column('netid', String(8), primary_key=True),
    Column('first_name', String(100)),
    Column('last_name', String(100)))

# Create Table in our database
query = students.create(engine, checkfirst=True)

In [ ]:
# Construct an insertion query
insert_query = students.insert().values(netid="bcongdo2",
                                        first_name="Ben",
                                        last_name="Congdon")

# The Query we will execute
print insert_query.compile()
# The parameters to this query
print insert_query.compile().params

In [ ]:
# Actually run the query
connection = engine.connect()
connection.execute(insert_query)

# Run some more insertions
connection.execute(students.insert(), [
        {'netid': 'abc1', 'first_name': 'john', 'last_name': 'doe'},
        {'netid': 'def2', 'first_name': 'jane', 'last_name': 'doe'}
    ])

In [ ]:
# Simple "SELECT"
s = select([students]) # Equivilant to "SELECT * FROM students"
result = connection.execute(s)
for i in result:
    print i

In [ ]:
# "SELECT" with parameters
# Equivilant to "SELECT netid FROM students WHERE first_name = "Ben""
s = select([students.c.netid]).where(students.c.first_name == "Ben")
for i in connection.execute(s):
    print i

Interested in learning more? Check out [this](http://docs.sqlalchemy.org/en/latest/core/tutorial.html) SQLAlchemy Expression Language tutorial.

### Example 2: SQLAlchemy as a Object Relational Mapping tool
When using an ORM, our SQL becomes even more "object" like, and we get to think less about the actual queries that are running. We are getting closer to a "in-language" database layer - but really, this is just providing an extra layer of abstraction.

In [ ]:
# Random SQLAlchemy magic
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Our Movie class
class Movie(Base):
    __tablename__ = "movies"
    id = Column(Integer, primary_key=True)
    title = Column(String)
    rating = Column(Float)
    duration_minutes = Column(Integer)
    release_date = Column(Date)
    
# Tell SQLAlchemy to create the table
Base.metadata.create_all(engine)

In [ ]:
# Now we can create and modify Movie objects
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

inception = Movie(title="Inception",
                  rating=8.8,
                  duration_minutes=148,
                  release_date=date(2010, 7, 16))
interstellar = Movie(title="Interstellar",
                     rating=8.6,
                     duration_minutes=169,
                     release_date=date(2014, 11, 4))

# Add our ORM objects to our session
session.add(inception)
session.add(interstellar)

# Tell SQLAlchemy to commit all changes to the database
session.commit()

In [ ]:
# Queries
# Get all movies
for movie in session.query(Movie):
    print movie.title, movie.id, movie.rating

In [ ]:
# Get movies with rating above 8.4
print session.query(Movie).filter(Movie.rating > 8.4)[0].title

Interested in learning more? Look [here](http://docs.sqlalchemy.org/en/rel_1_1/orm/tutorial.html) for a good SQLAlchemy ORM tutorial.

### Example 3: Pandas for SQL

In [ ]:
import pandas as pd

# Read in a CSV to a dataframe
df = pd.read_csv('airplane_crashes.csv', encoding='utf-8')

# Export a dataframe to a SQL table
df.to_sql('airplane_crashes', connection, if_exists='replace')

In [ ]:
# Load a SQL table into a dataframe
df_read = pd.read_sql('airplane_crashes', connection)
df_read

In [ ]:
# Execute SQL to get a dataframe
df_read = pd.read_sql('SELECT Date, Time, Type FROM airplane_crashes', connection)
df_read